# Analyse et nettoyage du jeu de données 

In [151]:
import pandas as pd
import numpy as np

### Chargement du fichier excel et transformation de chaque feuille en csv

In [152]:
# # Chemin vers le fichier Excel d'entrée
# excel_file_path = 'online_retail_II.xlsx'

# # Charger le fichier Excel en utilisant pandas
# xls = pd.ExcelFile(excel_file_path)

# # Parcourir chaque feuille de calcul du fichier Excel
# for sheet_name in xls.sheet_names:
#     # Charger la feuille de calcul en tant que dataframe
#     data_frame = xls.parse(sheet_name)
    
#     # Chemin vers le fichier CSV de sortie
#     csv_file_path = f'online_retail{sheet_name}.csv'
    
#     # Convertir le dataframe en fichier CSV
#     data_frame.to_csv(csv_file_path, index=False, encoding='utf-8')
    
# print('Conversion terminée.')


### Concaténation des deux feuilles en un fichier csv

In [153]:
# # Chemin vers le premier fichier CSV
# csv_file1 = 'online_retailYear 2009-2010.csv'

# # Chemin vers le deuxième fichier CSV
# csv_file2 = 'online_retailYear 2010-2011.csv'

# # Charger les deux fichiers CSV en utilisant pandas
# data_frame1 = pd.read_csv(csv_file1)
# data_frame2 = pd.read_csv(csv_file2)

# # Concaténer les deux dataframes
# concatenated_data = pd.concat([data_frame1, data_frame2])

# # Chemin vers le fichier CSV de sortie
# output_csv = 'online_retail.csv'

# # Enregistrer le dataframe concaténé en tant que fichier CSV
# concatenated_data.to_csv(output_csv, index=False)

# print('Concaténation terminée.')


## Nettoyage des données

In [154]:
df_ukretail = pd.read_csv('online_retail.csv')

In [155]:
import pandas as pd

# Charger le fichier CSV dans un DataFrame
df_ukretail = pd.read_csv('online_retail.csv')

# Convertir la colonne 'InvoiceDate' en type datetime
df_ukretail['InvoiceDate'] = pd.to_datetime(df_ukretail['InvoiceDate'])

# Filtrer les données pour la période de 2009-2010
df_2009_12_2010 = df_ukretail[(df_ukretail['InvoiceDate'] >= pd.to_datetime('2009-12-01')) & (df_ukretail['InvoiceDate'] <= pd.to_datetime('2010-12-31'))]

# Générer les nouveaux DataFrames en ajoutant un mois à chaque fois
df_2009_01_2011 = df_ukretail[(df_ukretail['InvoiceDate'] >= pd.to_datetime('2009-01-01')) & (df_ukretail['InvoiceDate'] <= pd.to_datetime('2011-01-31'))]
df_2009_02_2011 = df_ukretail[(df_ukretail['InvoiceDate'] >= pd.to_datetime('2009-01-01')) & (df_ukretail['InvoiceDate'] <= pd.to_datetime('2011-02-28'))]
df_2009_03_2011 = df_ukretail[(df_ukretail['InvoiceDate'] >= pd.to_datetime('2009-01-01')) & (df_ukretail['InvoiceDate'] <= pd.to_datetime('2011-03-31'))]
df_2009_04_2011 = df_ukretail[(df_ukretail['InvoiceDate'] >= pd.to_datetime('2009-01-01')) & (df_ukretail['InvoiceDate'] <= pd.to_datetime('2011-04-30'))]
df_2009_05_2011 = df_ukretail[(df_ukretail['InvoiceDate'] >= pd.to_datetime('2009-01-01')) & (df_ukretail['InvoiceDate'] <= pd.to_datetime('2011-05-31'))]
df_2009_06_2011 = df_ukretail[(df_ukretail['InvoiceDate'] >= pd.to_datetime('2009-01-01')) & (df_ukretail['InvoiceDate'] <= pd.to_datetime('2011-06-30'))]
df_2009_07_2011 = df_ukretail[(df_ukretail['InvoiceDate'] >= pd.to_datetime('2009-01-01')) & (df_ukretail['InvoiceDate'] <= pd.to_datetime('2011-07-31'))]
df_2009_08_2011 = df_ukretail[(df_ukretail['InvoiceDate'] >= pd.to_datetime('2009-01-01')) & (df_ukretail['InvoiceDate'] <= pd.to_datetime('2011-08-31'))]
df_2009_09_2011 = df_ukretail[(df_ukretail['InvoiceDate'] >= pd.to_datetime('2009-01-01')) & (df_ukretail['InvoiceDate'] <= pd.to_datetime('2011-09-30'))]
df_2009_10_2011 = df_ukretail[(df_ukretail['InvoiceDate'] >= pd.to_datetime('2009-01-01')) & (df_ukretail['InvoiceDate'] <= pd.to_datetime('2011-10-31'))]
df_2009_11_2011 = df_ukretail[(df_ukretail['InvoiceDate'] >= pd.to_datetime('2009-01-01')) & (df_ukretail['InvoiceDate'] <= pd.to_datetime('2011-11-30'))]
df_2009_12_2011 = df_ukretail[(df_ukretail['InvoiceDate'] >= pd.to_datetime('2009-01-01')) & (df_ukretail['InvoiceDate'] <= pd.to_datetime('2011-12-31'))]

In [156]:
df_2009_12_2010_copy = df_2009_12_2010.copy()
df_2009_01_2011_copy = df_2009_01_2011.copy()
df_2009_02_2011_copy = df_2009_02_2011.copy()
df_2009_03_2011_copy = df_2009_03_2011.copy()
df_2009_04_2011_copy = df_2009_04_2011.copy()
df_2009_05_2011_copy = df_2009_05_2011.copy()
df_2009_06_2011_copy = df_2009_06_2011.copy()
df_2009_07_2011_copy = df_2009_07_2011.copy()
df_2009_08_2011_copy = df_2009_08_2011.copy()
df_2009_09_2011_copy = df_2009_09_2011.copy()
df_2009_10_2011_copy = df_2009_10_2011.copy()
df_2009_11_2011_copy = df_2009_11_2011.copy()
df_2009_12_2011_copy = df_2009_12_2011.copy()

In [157]:
def clean_monthly_data_copy(df):
    # Supprimer les lignes où le client n'est pas identifié
    df = df.dropna(subset=['Customer ID'])

    # Convertir la colonne 'Customer ID' en type str
    df['Customer ID'] = df['Customer ID'].astype('str')

    # Supprimer les lignes avec un prix <= 0
    df = df[df['Price'] > 0]

    # Supprimer les lignes où le stockcode est composé uniquement de lettres
    df = df[~df['StockCode'].str.match(r'^[A-Za-z]')]

    df = df[df['Quantity']>0]
    
    # Ajouter une colonne 'total' qui représente le prix unitaire multiplié par la quantité
    df['total'] = df['Quantity'] * df['Price']

    # Group by 'Customer ID' et 'Invoice' et calculer les sommes
    df_gp_invoice = df.groupby(['Customer ID', 'Invoice']).sum().drop('Price', axis=1)

    # Group by 'Customer ID' et calculer les agrégats
    df_rfm = df.groupby('Customer ID').agg({
        'Invoice': 'nunique',
        'Quantity': 'sum',
        'InvoiceDate': 'max',
        'total': 'sum'
    })

    # Renommer les colonnes du DataFrame df_rfm
    df_rfm.columns = ['nombre_commandes', 'nombre_articles', 'date_derniere_commande', 'montant_total']
    df_rfm = df_rfm[['montant_total','nombre_articles']]
    df_rfm2 = df_rfm

    return df_rfm2

In [158]:
import pandas as pd
from datetime import datetime

def clean_monthly_data(df, df_rfm2):
    # Supprimer les lignes où le client n'est pas identifié
    df = df.dropna(subset=['Customer ID'])

    # Convertir la colonne 'Customer ID' en type str
    df['Customer ID'] = df['Customer ID'].astype('str')

    # Supprimer les lignes avec un prix <= 0
    df = df[df['Price'] > 0]

    # Supprimer les lignes où le stockcode est composé uniquement de lettres
    df = df[~df['StockCode'].str.match(r'^[A-Za-z]')]

    # Ajouter une colonne 'total' qui représente le prix unitaire multiplié par la quantité
    df['total'] = df['Quantity'] * df['Price']

    # Group by 'Customer ID' et 'Invoice' et calculer les sommes
    df_gp_invoice = df.groupby(['Customer ID', 'Invoice']).sum().drop('Price', axis=1)

    # Group by 'Customer ID' et calculer les agrégats
    df_rfm = df.groupby('Customer ID').agg({
        'Invoice': 'nunique',
        'Quantity': 'sum',
        'InvoiceDate': 'max',
        'total': 'sum'
    })

    # Renommer les colonnes du DataFrame df_rfm
    df_rfm.columns = ['nombre_commandes', 'nombre_articles', 'date_derniere_commande', 'montant_total']

    # Compter le nombre de commandes annulées par client
    cancelled_orders_count = df[df['Invoice'].str.startswith('C')].groupby('Customer ID')['Invoice'].nunique()

    # Fusionner avec le DataFrame df_rfm
    df_rfm = df_rfm.merge(cancelled_orders_count, how='left', on='Customer ID')
    df_rfm['Invoice'] = df_rfm['Invoice'].fillna(0)

    # Soustraire le nombre de commandes annulées du nombre total de commandes
    df_rfm['nombre_commandes'] = df_rfm['nombre_commandes'] - df_rfm['Invoice']
    df_rfm = df_rfm.rename(columns={'Invoice': 'transactions_annulation'})

    # Calculer la valeur de 'Recency' en utilisant une date de référence fixe
    date_max = str(df_rfm['date_derniere_commande'].max())
    date_format = '%Y-%m-%d %H:%M:%S'
    date_obj = datetime.strptime(date_max, date_format)
    df_rfm['date_derniere_commande'] = pd.to_datetime(df_rfm['date_derniere_commande'])
    df_rfm['nombre_jours_depuis_derniere_commande'] = (date_obj - df_rfm['date_derniere_commande']).dt.days

    # Charger le DataFrame df_rfm2
    df_rfm2 = pd.read_csv('df_rfm2.csv')
    df_rfm2['Customer ID'] = df_rfm2['Customer ID'].astype('str')
    
    df_rfm2.columns = ['Customer ID', 'montant_total','nombre_articles']

    # Fusionner avec le DataFrame df_rfm
    df_rfm = df_rfm.merge(df_rfm2, how='left', on='Customer ID')
    df_rfm = df_rfm.drop('nombre_articles_x', axis=1)

    # Calculer le pourcentage d'annulation de montant
    df_rfm['pourcentage_annulation_montant'] = ((df_rfm['montant_total_y'] - df_rfm['montant_total_x']) / df_rfm['montant_total_y']) * 100

    # Supprimer les lignes avec un montant total négatif
    df_rfm = df_rfm[df_rfm['montant_total_x'] >= 0]

    # Calculer le nombre moyen d'articles par commande
    df_rfm['nombre_articles_moyen_par_commande'] = df_rfm['nombre_articles_y'] / df_rfm['nombre_commandes']

    # Calculer le panier moyen par client
    df_rfm['panier_moyen'] = df_rfm['montant_total_y'] / df_rfm['nombre_commandes']

    # Charger le DataFrame df_country
    df_country = df[['Customer ID', 'Country']].drop_duplicates()

    # Fusionner avec le DataFrame df_rfm
    df_rfm = df_rfm.merge(df_country, how='left', on='Customer ID')
    df_rfm =df_rfm.drop_duplicates(subset='Customer ID')
    
    # Renommer les colonnes 
    df_rfm = df_rfm.rename(columns = {'NombreArticles_y':'nombre_articles_total','Customer ID':'numero_client','montant_total_x':'montant_effectif','montant_total_y':'montant_global','Country':'pays'})

    return df_rfm


In [159]:
df_list = [
    df_2009_12_2010,
    df_2009_01_2011,
    df_2009_02_2011,
    df_2009_03_2011,
    df_2009_04_2011,
    df_2009_05_2011,
    df_2009_06_2011,
    df_2009_07_2011,
    df_2009_08_2011,
    df_2009_09_2011,
    df_2009_10_2011,
    df_2009_11_2011,
    df_2009_12_2011
]

df_list_copy = [
    df_2009_12_2011_copy,
    df_2009_01_2011_copy,
    df_2009_02_2011_copy,
    df_2009_03_2011_copy,
    df_2009_04_2011_copy,
    df_2009_05_2011_copy,
    df_2009_06_2011_copy,
    df_2009_07_2011_copy,
    df_2009_08_2011_copy,
    df_2009_09_2011_copy,
    df_2009_10_2011_copy,
    df_2009_11_2011_copy,
    df_2009_12_2011_copy
]

for i in range(len(df_list)):
    df_list_copy[i] = clean_monthly_data_copy(df_list_copy[i])
    df_list[i] = clean_monthly_data(df_list[i], df_list_copy[i])

df_2009_12_2010 = df_list[0]
df_2009_01_2011 = df_list[1]
df_2009_02_2011 = df_list[2]
df_2009_03_2011 = df_list[3]
df_2009_04_2011 = df_list[4]
df_2009_05_2011 = df_list[5]
df_2009_06_2011 = df_list[6]
df_2009_07_2011 = df_list[7]
df_2009_08_2011 = df_list[8]
df_2009_09_2011 = df_list[9]
df_2009_10_2011 = df_list[10]
df_2009_11_2011 = df_list[11]
df_2009_12_2011 = df_list[12]

/tmp/ipykernel_64280/304799826.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Customer ID'] = df['Customer ID'].astype('str')
/tmp/ipykernel_64280/304799826.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_gp_invoice = df.groupby(['Customer ID', 'Invoice']).sum().drop('Price', axis=1)
/tmp/ipykernel_64280/1801774541.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [160]:
def iqr_outlier(column, lower_multiplier=1.5, upper_multiplier=1.5):
    q1 = column.quantile(0.05)
    q3 = column.quantile(0.95)
    iqr = q3 - q1
    lower_threshold = q1 - lower_multiplier * iqr
    upper_threshold = q3 + upper_multiplier * iqr
    outliers = (column < lower_threshold) | (column > upper_threshold)
    return outliers


def remove_outliers(df, lower_multiplier=1.5, upper_multiplier=1.5):
    df_no_outliers = df.copy()
    for column in df.select_dtypes(include=[np.number]).columns:
        outliers = iqr_outlier(df[column], lower_multiplier, upper_multiplier)
        df_no_outliers = df_no_outliers[~outliers]
    return df_no_outliers


In [161]:
df_no_outliers_2009_12_2010 = remove_outliers(df_2009_12_2010)
df_no_outliers_2009_01_2011 = remove_outliers(df_2009_01_2011)
df_no_outliers_2009_02_2011 = remove_outliers(df_2009_02_2011)
df_no_outliers_2009_03_2011 = remove_outliers(df_2009_03_2011)
df_no_outliers_2009_04_2011 = remove_outliers(df_2009_04_2011)
df_no_outliers_2009_05_2011 = remove_outliers(df_2009_05_2011)
df_no_outliers_2009_06_2011 = remove_outliers(df_2009_06_2011)
df_no_outliers_2009_07_2011 = remove_outliers(df_2009_07_2011)
df_no_outliers_2009_08_2011 = remove_outliers(df_2009_08_2011)
df_no_outliers_2009_09_2011 = remove_outliers(df_2009_09_2011)
df_no_outliers_2009_10_2011 = remove_outliers(df_2009_10_2011)
df_no_outliers_2009_11_2011 = remove_outliers(df_2009_11_2011)
df_no_outliers_2009_12_2011 = remove_outliers(df_2009_12_2011)


/tmp/ipykernel_64280/33839429.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_no_outliers = df_no_outliers[~outliers]
/tmp/ipykernel_64280/33839429.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_no_outliers = df_no_outliers[~outliers]
/tmp/ipykernel_64280/33839429.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_no_outliers = df_no_outliers[~outliers]
/tmp/ipykernel_64280/33839429.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_no_outliers = df_no_outliers[~outliers]
/tmp/ipykernel_64280/33839429.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_no_outliers = df_no_outliers[~outliers]
/tmp/ipykernel_64280/33839429.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_no_outliers = df_no_outliers[~outliers]
/tmp/ipykernel_64280/33839429.py:15: UserWarning: Bo

In [162]:
df_no_outliers_2009_12_2010.to_csv('df_no_outliers_2009_12_2010.csv', index=False)
df_no_outliers_2009_01_2011.to_csv('df_no_outliers_2009_01_2011.csv', index=False)
df_no_outliers_2009_02_2011.to_csv('df_no_outliers_2009_02_2011.csv', index=False)
df_no_outliers_2009_03_2011.to_csv('df_no_outliers_2009_03_2011.csv', index=False)
df_no_outliers_2009_04_2011.to_csv('df_no_outliers_2009_04_2011.csv', index=False)
df_no_outliers_2009_05_2011.to_csv('df_no_outliers_2009_05_2011.csv', index=False)
df_no_outliers_2009_06_2011.to_csv('df_no_outliers_2009_06_2011.csv', index=False)
df_no_outliers_2009_07_2011.to_csv('df_no_outliers_2009_07_2011.csv', index=False)
df_no_outliers_2009_08_2011.to_csv('df_no_outliers_2009_08_2011.csv', index=False)
df_no_outliers_2009_09_2011.to_csv('df_no_outliers_2009_09_2011.csv', index=False)
df_no_outliers_2009_10_2011.to_csv('df_no_outliers_2009_10_2011.csv', index=False)
df_no_outliers_2009_11_2011.to_csv('df_no_outliers_2009_11_2011.csv', index=False)
df_no_outliers_2009_12_2011.to_csv('df_no_outliers_2009_12_2011.csv', index=False)

In [163]:
df_no_outliers_2009_12_2011.head()

,numero_client,nombre_commandes,date_derniere_commande,montant_effectif,transactions_annulation,nombre_jours_depuis_derniere_commande,montant_global,nombre_articles_y,pourcentage_annulation_montant,nombre_articles_moyen_par_commande,panier_moyen,pays
1,12347.0,8.0,2011-12-07 15:52:00,5633.32,0.0,1,5633.32,3286.0,0.000000e+00,410.750000,704.165,Iceland
2,12348.0,5.0,2011-09-25 13:13:00,1658.40,0.0,74,1658.40,2704.0,1.371042e-14,540.800000,331.680,Finland
3,12349.0,3.0,2011-11-21 09:51:00,3654.54,1.0,18,3678.69,1621.0,6.564837e-01,540.333333,1226.230,Italy
4,12350.0,1.0,2011-02-02 16:01:00,294.40,0.0,309,294.40,196.0,0.000000e+00,196.000000,294.400,Norway
5,12351.0,1.0,2010-11-29 15:23:00,300.93,0.0,374,300.93,261.0,0.000000e+00,261.000000,300.930,Unspecified


In [164]:
import matplotlib.pyplot as plt

plt.hist(df_no_outliers_2009_12_2010.NombreArticlesMoyenParCommande)

AttributeError: 'DataFrame' object has no attribute 'NombreArticlesMoyenParCommande'